## installation

In [1]:
# If not installed the 1st time, install it
# !pip install mistral_inference

## download model

In [2]:

from huggingface_hub import snapshot_download
from pathlib import Path
import os

mistral_models_path = Path.home().joinpath(os.getcwd(), 'mistral_models', '7B-Instruct-v0.3')
if not os.path.exists(mistral_models_path):
    mistral_models_path.mkdir(parents=True, exist_ok=True)

    snapshot_download(repo_id="mistralai/Mistral-7B-Instruct-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)


/home/acorn/anaconda3/envs/opencompass/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Model

In [3]:
from mistral_common.protocol.instruct.tool_calls import Function, Tool
from mistral_inference.model import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage, AssistantMessage, ToolMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest


tokenizer = MistralTokenizer.from_file(f"{mistral_models_path}/tokenizer.model.v3")
model = Transformer.from_folder(mistral_models_path)




## Chat Completion with functions

In [4]:
from utils import functions
from mistral_common.protocol.instruct.messages import UserMessage, AssistantMessage, ToolMessage

tools = [ Tool(function=Function(name=f["function"]["name"], description=f["function"]["description"], parameters=f["function"]["parameters"])) for f in functions]

## collected from https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3#function-calling

def run_completion(msgs):
    completion_request = ChatCompletionRequest(
        tools=tools,
        messages=msgs,
    )

    tokens = tokenizer.encode_chat_completion(completion_request).tokens

    out_tokens, _ = generate([tokens], model, max_tokens=8000, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
    result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

    print(result)
    return result

mock_funtion_call_res = {
    "list_files":  "utils.py, benchmark.py, readme.md",
    "create_3d_model": "a 3d model is created",
    "orderUmbrella": "a ten dollar umbrella has been ordered",
    "generate_password": "password 'k2jh1312' generated. Keep it safe."
}

In [14]:
import json

with open('questions.json', 'r') as file:
    data = json.load(file)
    questions = [d["user_query"] for d in data['queries']]

for i,q in enumerate(questions):
    print(f"================\nQESTION: {q}\n")
    msgs = []
    msgs.append(UserMessage(content=q))
    res = run_completion(msgs)
    try: 
        loaded_res = json.loads(res)
        msgs.append(AssistantMessage(content=res))
        func_res = []
        for l in loaded_res:
            func_name = l["name"]
            func_res.append(mock_funtion_call_res[func_name])
        msgs.append(ToolMessage(content=str(func_res), tool_call_id="aaaabbbbc"))
        run_completion(msgs)
            
    except Exception as e:
        print(f"Error: {e}")

QESTION: What is the distance between San Francisco and Cupertino by driving and by air from both directions?

To calculate the distance between San Francisco and Cupertino, I can use the `calculate_distance` function. However, I don't have real-time traffic or flight data. For demonstration purposes, let's assume we're using driving and flying as the modes of transportation.

First, let's calculate the driving distance from San Francisco to Cupertino:

```javascript
calculate_distance({
  origin: "San Francisco",
  destination: "Cupertino",
  mode: "driving"
})
```

Next, let's calculate the driving distance from Cupertino to San Francisco:

```javascript
calculate_distance({
  origin: "Cupertino",
  destination: "San Francisco",
  mode: "driving"
})
```

Now, let's calculate the flying distance from San Francisco to Cupertino:

```javascript
calculate_distance({
  origin: "San Francisco",
  destination: "Cupertino",
  mode: "flying"
})
```

Finally, let's calculate the flying distanc